In [2]:
import pandas as pd
import numpy as np
from math import radians
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
from sklearn.metrics import auc, roc_curve, confusion_matrix
import pickle

In [3]:
df = pd.read_parquet('data/BMTC.parquet.gzip', engine='pyarrow')  # Speeds @ (loc & time)
dfInput = pd.read_csv('data/Input.csv')                           # Input DF
dfGroundTruth = pd.read_csv('data/GroundTruth.csv')               # Corresponding Timetaken (O/P)
df

,BusID,Latitude,Longitude,Speed,Timestamp
0,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:02
1,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:12
2,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:22
3,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:32
4,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:42
...,...,...,...,...,...
6014941,150813395,13.097612,77.565689,30.0,2019-08-01 18:59:20
6014942,150813395,13.097538,77.564873,34.0,2019-08-01 18:59:30
6014943,150813395,13.097462,77.564049,38.0,2019-08-01 18:59:40
6014944,150813395,13.097604,77.563217,21.0,2019-08-01 18:59:50


In [4]:
df = df.drop_duplicates(subset=["Latitude", "Longitude"], keep='first')
df.reset_index(drop=True, inplace=True)
df

,BusID,Latitude,Longitude,Speed,Timestamp
0,150212121,13.074558,77.445549,0.0,2019-08-01 07:00:02
1,150212121,13.074113,77.445282,21.0,2019-08-01 07:25:26
2,150212121,13.074060,77.445267,0.0,2019-08-01 07:25:36
3,150212121,13.069714,77.446480,22.0,2019-08-01 07:31:04
4,150212121,13.069661,77.446579,14.0,2019-08-01 07:31:14
...,...,...,...,...,...
3751330,150813395,13.097612,77.565689,30.0,2019-08-01 18:59:20
3751331,150813395,13.097538,77.564873,34.0,2019-08-01 18:59:30
3751332,150813395,13.097462,77.564049,38.0,2019-08-01 18:59:40
3751333,150813395,13.097604,77.563217,21.0,2019-08-01 18:59:50


In [56]:
# df.to_csv("data/speeds.csv")

In [5]:
df['Speed'].value_counts()

0.0      1095109
20.0       90576
21.0       89606
22.0       88030
23.0       85374
          ...   
112.0          1
104.0          1
115.0          1
116.0          1
106.0          1
Name: Speed, Length: 113, dtype: int64

In [25]:
for i in range(len(df)):
    if(df.loc[i, 'Speed'] == 25.0):
        print('%.6f' % df.loc[i, 'Latitude'],'%.6f' % df.loc[i, 'Longitude'], df.loc[i, 'Speed'])
    if i == 1000:
        break

13.067086 77.451355 25.0
13.065458 77.454155 25.0
13.058535 77.467628 25.0
13.053075 77.485611 25.0
13.042361 77.515289 25.0
13.041442 77.516991 25.0
13.026783 77.543869 25.0
13.025930 77.545357 25.0
13.023088 77.550278 25.0
12.991252 77.574890 25.0
12.986321 77.574554 25.0
12.980680 77.577888 25.0
12.980459 77.579285 25.0
12.972924 77.581238 25.0
13.007264 77.571198 25.0
13.010179 77.570641 25.0
13.034695 77.528709 25.0
13.045845 77.504982 25.0
13.054825 77.473846 25.0
13.044542 77.509506 25.0
13.042889 77.513504 25.0
13.036730 77.524559 25.0
13.015430 77.553864 25.0
13.016495 77.561157 25.0
13.015223 77.563370 25.0
13.012750 77.567627 25.0
12.998147 77.569382 25.0
12.982803 77.574600 25.0
13.019767 77.552879 25.0
13.035229 77.526909 25.0
13.051470 77.490204 25.0
13.042819 77.513733 25.0
13.037107 77.523758 25.0
13.025547 77.546173 25.0
13.019224 77.553238 25.0
13.015483 77.554649 25.0
13.014243 77.565590 25.0
13.006676 77.571297 25.0
13.018323 77.558624 25.0
13.020060 77.552711 25.0


In [5]:
df['Latitude'] = df['Latitude'].apply(radians)
df['Longitude'] = df['Longitude'].apply(radians)

C:\Users\RAZAK\AppData\Local\Temp\ipykernel_27768\562795013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'] = df['Latitude'].apply(radians)
C:\Users\RAZAK\AppData\Local\Temp\ipykernel_27768\562795013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'] = df['Longitude'].apply(radians)


Training KNNRegressor Model

In [37]:
X = df[['Latitude', 'Longitude']].values
y = df['Speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
nbrs = KNeighborsRegressor(weights='distance', metric='haversine', n_neighbors=21)
nbrs.fit(X_train, y_train)
y_pred = nbrs.predict(X_test)

In [38]:
# rad = radians(77.445549)
# rad
nbrs.predict([[radians(12.963536),radians(77.583687)]])

array([57.42867335])

In [16]:
pickle.dump(nbrs, open('21nbrs.sav', 'wb'))

Use Model

In [6]:
nbrs1 = pickle.load(open('21nbrs(5.19,67.91,8.24).sav', 'rb'))

In [36]:
nbrs1.predict([[math.radians(12.986321),radians( 77.574554)]])

array([19.84669122])

In [39]:
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("RMSE:", math.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 6.823817717788871
MSE: 88.35952772744791
RMSE: 9.399974879086002


In [39]:
# Evaluation
fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=True)
auc_score = round(auc(fpr, tpr), 2)
threshold = 0.5
print("AUC: {}".format(auc_score))

AUC: nan


c:\Users\RAZAK\Desktop\Github\CNI_Hackathon-main\venv\lib\site-packages\sklearn\metrics\_ranking.py:1018: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Train data

In [19]:
x_pred = nbrs.predict(X_train)
# "MAE:", mean_absolute_error(y_test, y_pred)

In [23]:
print("MAE:", mean_absolute_error(y_train, x_pred))
print("MSE:", mean_squared_error(y_train, x_pred))
print("RMSE:", math.sqrt(mean_squared_error(y_train, x_pred)))

MAE: 0.8095689499939684
MSE: 5.969410741850648
RMSE: 2.4432377579455196


In [3]:
# data_speeds = np.array(df)
# len(data_speeds)

In [4]:
# mp_locs_speed = {}
# for i in data_speeds:
#     lat = i[1]
#     lon = i[2] 
#     speed = i[3] 
#     if (lat,lon) in mp_locs_speed:
#         mp_locs_speed[(lat,lon)] = float(mp_locs_speed[(lat,lon)] + speed) / 2
#     else:
#         mp_locs_speed[(lat,lon)] = speed

In [5]:
# mp_locs_speed[(13.097604,77.563217)]

In [6]:
# len(mp_locs_speed)

In [7]:
dfInput = dfInput.loc[:, ~dfInput.columns.str.contains('^Unnamed')]
dfInput


,Source_Lat,Source_Long,Dest_Lat,Dest_Long
0,12.941644,77.557335,12.942002,77.551605
1,12.845487,77.662079,12.845881,77.667892
2,12.973492,77.622871,12.957303,77.621246
3,12.819298,77.688995,12.814241,77.692986
4,12.973240,77.615402,13.016170,77.627800
...,...,...,...,...
1200,13.097792,77.591736,13.154475,77.568077
1201,12.794702,77.624046,12.863790,77.616608
1202,12.922270,77.743591,12.983501,77.752258
1203,12.918303,77.589500,12.923568,77.655655


In [8]:
dfGroundTruth = dfGroundTruth.loc[:, ~dfGroundTruth.columns.str.contains('^Unnamed')]
dfGroundTruth

,TT
0,2.833333
1,1.500000
2,21.250000
3,2.000000
4,35.733333
...,...
1200,27.233333
1201,26.750000
1202,53.600000
1203,59.616667


In [9]:
df_combined = pd.concat([dfInput, dfGroundTruth], axis=1)
df_combined

,Source_Lat,Source_Long,Dest_Lat,Dest_Long,TT
0,12.941644,77.557335,12.942002,77.551605,2.833333
1,12.845487,77.662079,12.845881,77.667892,1.500000
2,12.973492,77.622871,12.957303,77.621246,21.250000
3,12.819298,77.688995,12.814241,77.692986,2.000000
4,12.973240,77.615402,13.016170,77.627800,35.733333
...,...,...,...,...,...
1200,13.097792,77.591736,13.154475,77.568077,27.233333
1201,12.794702,77.624046,12.863790,77.616608,26.750000
1202,12.922270,77.743591,12.983501,77.752258,53.600000
1203,12.918303,77.589500,12.923568,77.655655,59.616667
